In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import  classification_report
import os

In [11]:
file = "/media/hamza/289875229874EFA4/WORK/annotation/done/txt/04_Mon-31-01-22-Fri-07-02-22/gen_ner_35k_farrukh.txt" 
df = pd.read_fwf(file, header=None, usecols=[0])

In [34]:
# mask = (len(df[0].lower().split()) <= 30)
# df = df.loc[mask]
# df.size

In [4]:
# Change column "Score" values 
## 1-5(1/2 => negative, 3 => neutral, 4/5 => positive) to
## 0-1(negative/positive)

def partition(x):
    """
    Score>3 a positive rating,
    Score<3 a negative rating.
    """
    if x == 'Positive':
        return 0
    elif x == 'Negative':
        return 1
    elif x == 'Neutral':
        return 2
    elif x == 'Mixed':
        return 3

# changing reviews with score less than 3 to be positive and vice-versa
actualScore = df['Sentiment']
positiveNegative = actualScore.map(partition) 
df['Sentiment'] = positiveNegative

print("Number of data points in our data", df.shape)
df.head()

Number of data points in our data (734, 2)


,Text,Sentiment
0,Tabish Rafiq,NaN
3,Abir Qureshi,NaN
5,Jamil Khan,NaN
6,Javed Afridi,NaN
7,Malik Bilal,NaN


In [5]:
df1 = df[df['Sentiment']==0]
df2 = df[df['Sentiment']==1]
df4 = df[df['Sentiment']==2]

df = pd.concat([df1, df2,df4])# df3, df4])
df['Sentiment'] = df['Sentiment'].astype('int')
df = df.dropna()

df.head()

,Text,Sentiment
27,"Yours truly,",0
540,Thank you.,0
590,Allah apki madad karay .,0
709,I hope Allah do best for you,0
755,May u recover soon,0


In [6]:
def decontracted(word):
    # https://www.kaggle.com/smat26/sentiment-analysis-on-roman-urdu
    import re
    import bs4 as bs
    # import re

    word = re.sub(r'ain$', r'ein', word)
    word = re.sub(r'ai', r'ae', word)
    word = re.sub(r'ay$', r'e', word)
    word = re.sub(r'ey$', r'e', word)
    word = re.sub(r'ie$', r'y', word)
    word = re.sub(r'^es', r'is', word)
    word = re.sub(r'a+', r'a', word)
    word = re.sub(r'j+', r'j', word)
    word = re.sub(r'd+', r'd', word)
    word = re.sub(r'u', r'o', word)
    word = re.sub(r'o+', r'o', word)
    word = re.sub(r'ee+', r'i', word)
    if not re.match(r'ar', word):
        word = re.sub(r'ar', r'r', word)
        word = re.sub(r'iy+', r'i', word)
        word = re.sub(r'ih+', r'eh', word)
        word = re.sub(r's+', r's', word)
    if re.search(r'[rst]y', 'word') and word[-1] != 'y':
        word = re.sub(r'y', r'i', word)
    if re.search(r'[bcdefghijklmnopqrtuvwxyz]i', word):
        word = re.sub(r'i$', r'y', word)
    if re.search(r'[acefghijlmnoqrstuvwxyz]h', word):
        word = re.sub(r'h', '', word)
        word = re.sub(r'k', r'q', word)
        
    word = re.sub(r'\#\.', '', word)
    word = re.sub(r'\n', '', word)
    word = re.sub(r',', '', word)
    word = re.sub(r'\-', ' ', word)
    word = re.sub(r'\.', '', word)
    word = re.sub(r'\\', ' ', word)
    word = re.sub(r'\\x\.+', '', word)
    word = re.sub(r'\d', '', word)
    word = re.sub(r'^_.', '', word)
    word = re.sub(r'_', ' ', word)
    word = re.sub(r'^ ', '', word)
    word = re.sub(r' $', '', word)
    word = re.sub(r'\?', '', word)
    
    # Remove HTML tags
    word = re.sub(r"http\S+", "", word)
    word = bs.BeautifulSoup(word, 'lxml').get_text()

    # Remove numeric values including alphanueric value
    word = re.sub("\S*\d\S*", "", word).strip()

    # Remove special chars
    word = re.sub('[^A-Za-z]+', ' ', word)
    return word.lower()
    
def lemmatizing(text):
    from nltk.stem import WordNetLemmatizer
    from nltk import word_tokenize

    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = lemmatizer.lemmatize(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)

def text_preprocessing(df_text):
    from tqdm import tqdm
    
    preprocessed_msg = []
    # tqdm is for printing the status bar
    for sentance in tqdm(df_text.values):
        # Remove undesired chars
        sentance = decontracted(sentance)
        # sentance = lemmatizing(sentance)

        # Stop words
        ## https://deepnote.com/@umaimakhh/Sentimental-Analysis-on-Roman-Urdu-CYPLZfCxTjWGLxWzS2BaDg
        ## some words like Mein - Ma - Maa
        stopwords= ['uski', 'krna', 'kch', 'gai', 'raha', 'you', 'le', 'ki', 'krta', 'ga', 'jo', 'haan', 'de',
                    'hun', 'ho', 'meri', 'is', 'jee', 'lg', 'dain', 'hoa', 'jati', 'jana', 'rahi', 'kar', 'tou',
                    'photo', 'rha', 'ayi', 'wala', 'lye', 'kis', 'sy', 'ne', 'wohi', 'gy', 'gi', 'ny', 'hi', 'ab', 'gaya',
                    'je', 'hotay', 'jata', 'pe', 'hota', 'aye', 'me', 'tha', 'rahay', 'rhy', 'gya', 'us', 'thay', 'bhi',
                    'liye', 'sent', 'hai', 'aur', 'thi', 'jye', 'dono', 'phr', 'yehi', 'to', 'na', 'main', 'abi', 'hum',
                    'koi', 'hova', 'hain', 'ja', 'un', 'par', 'kafi', 'kr', 'hy', 'karna', 'h', 'krny', 'wo', 'kay', 'acha',
                    'mil', 'jb', 'ya', 'kya', 'ai', 'kuch', 'se', 'woh', 'thy', 'ap', 'houn', 'hay', 'sb', 'ka', 'he', 'han',
                    'chal', 'in', 'waisay', 'aese', 'hui', 'or', 'tu', 'ko', 'yeh']# , 'nai'

        
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)

        # Append processed sentense to list
        preprocessed_msg.append(sentance.strip())
        
    return preprocessed_msg
        
df['processed_text'] = text_preprocessing(df['Text'].astype("str"))
df.drop('Text', axis=1, inplace=True)
df.head()

100%|██████████████████████████████████████████| 88/88 [00:00<00:00, 134.61it/s]


,Sentiment,processed_text
27,0,yors troly
540,0,thank yo
590,0,alla apqi madad qray
709,0,i ope alla do best for yo
755,0,may o recover son


In [14]:
vect = joblib.load('data/tfidf_model.model')
data = vect.transform(df['processed_text'])
data = data.toarray()
print(data.shape)



(88, 2000)


0.0

In [8]:
votting_model = joblib.load('data/votting_model.model')

y_pred = votting_model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.11      0.25      0.15         8
           1       0.32      0.23      0.27        35
           2       0.50      0.49      0.49        45

    accuracy                           0.36        88
   macro avg       0.31      0.32      0.30        88
weighted avg       0.39      0.36      0.37        88



In [11]:
stacking_model = joblib.load('data/stacking_model.model')

y_pred = stacking_model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.10      0.25      0.14         8
           1       0.34      0.29      0.31        35
           2       0.50      0.42      0.46        45

    accuracy                           0.35        88
   macro avg       0.31      0.32      0.30        88
weighted avg       0.40      0.35      0.37        88



In [12]:
svm_linear_model = joblib.load("data/svm_linear.model")
y_pred = svm_linear_model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.08      0.25      0.12         8
           1       0.55      0.31      0.40        35
           2       0.64      0.62      0.63        45

    accuracy                           0.47        88
   macro avg       0.42      0.40      0.38        88
weighted avg       0.55      0.47      0.49        88



In [13]:
model = joblib.load("data/svm_nonLinear.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.21      0.88      0.34         8
           1       0.53      0.23      0.32        35
           2       0.55      0.49      0.52        45

    accuracy                           0.42        88
   macro avg       0.43      0.53      0.39        88
weighted avg       0.51      0.42      0.42        88



In [14]:
nb = joblib.load("data/nb.model")
y_pred = nb.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.14      0.62      0.23         8
           1       0.65      0.43      0.52        35
           2       0.67      0.44      0.53        45

    accuracy                           0.45        88
   macro avg       0.49      0.50      0.43        88
weighted avg       0.61      0.45      0.50        88



In [15]:
model = joblib.load("data/lr.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.18      0.62      0.28         8
           1       0.56      0.40      0.47        35
           2       0.69      0.53      0.60        45

    accuracy                           0.49        88
   macro avg       0.47      0.52      0.45        88
weighted avg       0.59      0.49      0.52        88



In [16]:
model = joblib.load("data/dt.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.38      0.75      0.50         8
           1       0.68      0.37      0.48        35
           2       0.68      0.80      0.73        45

    accuracy                           0.62        88
   macro avg       0.58      0.64      0.57        88
weighted avg       0.65      0.62      0.61        88



In [17]:
model = joblib.load("data/knn.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.19      0.50      0.28         8
           1       0.50      0.31      0.39        35
           2       0.62      0.62      0.62        45

    accuracy                           0.49        88
   macro avg       0.44      0.48      0.43        88
weighted avg       0.53      0.49      0.50        88



In [18]:
model = joblib.load("data/rf.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.22      0.62      0.32         8
           1       0.50      0.17      0.26        35
           2       0.57      0.67      0.61        45

    accuracy                           0.47        88
   macro avg       0.43      0.49      0.40        88
weighted avg       0.51      0.47      0.44        88



## X_train

In [19]:
vect = joblib.load('tfidf_X_train.model')
data = vect.transform(df['processed_text'])
data = data.toarray()
print(data.shape)

(88, 2000)


In [20]:
votting_model = joblib.load('data/votting_model.model')

y_pred = votting_model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.05      0.12      0.07         8
           1       0.36      0.14      0.20        35
           2       0.48      0.58      0.53        45

    accuracy                           0.36        88
   macro avg       0.30      0.28      0.27        88
weighted avg       0.39      0.36      0.36        88



In [21]:
stacking_model = joblib.load('data/stacking_model.model')

y_pred = stacking_model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.09      0.25      0.13         8
           1       0.46      0.17      0.25        35
           2       0.48      0.56      0.52        45

    accuracy                           0.38        88
   macro avg       0.34      0.33      0.30        88
weighted avg       0.44      0.38      0.37        88



In [22]:
svm_linear_model = joblib.load("data/svm_linear.model")
y_pred = svm_linear_model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.11      0.38      0.17         8
           1       0.50      0.31      0.39        35
           2       0.53      0.44      0.48        45

    accuracy                           0.39        88
   macro avg       0.38      0.38      0.34        88
weighted avg       0.48      0.39      0.42        88



In [23]:
model = joblib.load("data/svm_nonLinear.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.25      0.75      0.38         8
           1       0.71      0.34      0.46        35
           2       0.60      0.62      0.61        45

    accuracy                           0.52        88
   macro avg       0.52      0.57      0.48        88
weighted avg       0.61      0.52      0.53        88



In [24]:
nb = joblib.load("data/nb.model")
y_pred = nb.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.25      0.50      0.33         8
           1       0.57      0.46      0.51        35
           2       0.55      0.53      0.54        45

    accuracy                           0.50        88
   macro avg       0.46      0.50      0.46        88
weighted avg       0.53      0.50      0.51        88



In [25]:
model = joblib.load("data/lr.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.25      0.62      0.36         8
           1       0.62      0.46      0.52        35
           2       0.60      0.56      0.57        45

    accuracy                           0.52        88
   macro avg       0.49      0.55      0.49        88
weighted avg       0.57      0.52      0.53        88



In [26]:
model = joblib.load("data/dt.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.33      0.75      0.46         8
           1       0.58      0.40      0.47        35
           2       0.63      0.64      0.64        45

    accuracy                           0.56        88
   macro avg       0.52      0.60      0.52        88
weighted avg       0.58      0.56      0.56        88



In [27]:
model = joblib.load("data/knn.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.25      0.75      0.38         8
           1       0.50      0.31      0.39        35
           2       0.57      0.53      0.55        45

    accuracy                           0.47        88
   macro avg       0.44      0.53      0.44        88
weighted avg       0.51      0.47      0.47        88



In [28]:
model = joblib.load("data/rf.model")
y_pred = model.predict(data)
print(classification_report(df['Sentiment'], y_pred))

              precision    recall  f1-score   support

           0       0.28      0.62      0.38         8
           1       0.79      0.43      0.56        35
           2       0.63      0.71      0.67        45

    accuracy                           0.59        88
   macro avg       0.56      0.59      0.54        88
weighted avg       0.66      0.59      0.60        88



In [8]:
## Testing  Boosting

# votting_model = joblib.load('data/votting_model.model')
# stacking_model = joblib.load('data/stacking_model.model')
# svm_linear_model = joblib.load("data/svm_linear.model")
# svn_nonlinear_model = joblib.load("data/svm_nonLinear.model")
# nb_model = joblib.load("data/nb.model")
lr_model = joblib.load("data/lr.model")
dt_model = joblib.load("data/dt.model")
# knn_model = joblib.load("data/knn.model")
rf_model = joblib.load("data/rf.model")

In [10]:
models = [
    # votting_model,
    #       stacking_model,
    #       svm_linear_model,
    #       svn_nonlinear_model, 
    #       nb_model, 
          lr_model, 
          dt_model,
          # knn_model, 
          rf_model]

In [16]:
# for d in data[0][0]:
pred_list = []
for model in models:
    pred_list.append(model.predict(data))
max(set(pred_list), key=pred_list.count)

TypeError: unhashable type: 'numpy.ndarray'